<a href="https://colab.research.google.com/github/seph0916/seph-solo-machine/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A05%EB%8B%A8%EC%9B%90_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 교차 검증과 그리드 서치
# 테스트 세트에 맞는 모델을 만드는것이 아닌 일반적일때 잘 예측할수 있는 모델을 만들고싶음.
# 테스트 세트를 사용하지 않으면 모델이 과대,과소 적합인지 판단하기 어려움
# 테스트 세트를 사용하지 않고 측정하는 방법은 훈련세트를 또 나누는것-> 검증세트

In [3]:
# 와인 데이터 읽기
import pandas as pd
wine= pd.read_csv('https://bit.ly/wine-date')

In [4]:
# input data ,target data 나누기
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [7]:
# train data를 이용하여 또다시 훈련세트와 검증세트로 나눔
sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [9]:
print(wine.shape,sub_input.shape,val_input.shape)

(6497, 4) (4157, 3) (1040, 3)


In [16]:
# train data중 80퍼센트(80퍼 중에 80퍼)에 해당하는 데이터를 통해 모델을 만들고 확인해보기
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))
# 예상했던것과 동일하게 훈련세트에만 과대적합되어있는 경향이 있음

0.9971133028626413
0.864423076923077


교차검증: 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복 , 그다음 이 점수를 평균하여 최종 검증 점수를 얻는 방법
(교차 검증 함수 cross_validate())

---

*   3-폴드 교차 검증 이란?
: 훈련세트를 3부분으로 나눠 교차검증을 수행하는것 ex) 훈련 셋 훈련 셋 검증 셋 ,훈련 셋 검증 셋 훈련 셋 , 검증 셋 훈련 셋 훈련 셋

In [30]:
# 교차 검증 함수 사용
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)
# fit time , score time : 모델을 훈련하는 시간과 검증하는 시간
# test score : 검증 폴드의 점수
# corss_validate() 함수는 훈련세트를 섞어 폴드를 나누지 않음

{'fit_time': array([0.01156163, 0.01003981, 0.0075233 , 0.00739312, 0.00800228]), 'score_time': array([0.0016849 , 0.00117397, 0.0009985 , 0.00096416, 0.00103545]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [31]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [33]:
# StratifiedKFold 모델 만들어보기
from sklearn.model_selection import StratifiedKFold
# 앞서 수행한 교차검증 코드
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [35]:
# 10-폴드 교차 검증 수행 모델
# n_split 몇 폴드 교차 검증을 할지 정할수있음
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
#하이퍼 파라미터 (사용자 지정 파라미터) 튜닝